In [1]:
import boto3
s3 = boto3.client('s3')
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '8Y1AHKN9AYQ17WME',
  'HostId': '0fLfWrSish80BVEc94Z+1M1pmPILY4GUwn2cvDBnwjJzwPNdgdudiYi7hK9FMJwXyn4d1NKi5/M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '0fLfWrSish80BVEc94Z+1M1pmPILY4GUwn2cvDBnwjJzwPNdgdudiYi7hK9FMJwXyn4d1NKi5/M=',
   'x-amz-request-id': '8Y1AHKN9AYQ17WME',
   'date': 'Sat, 28 Feb 2026 19:09:19 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'ananya-website.com',
   'CreationDate': datetime.datetime(2025, 4, 21, 1, 16, 10, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::ananya-website.com'},
  {'Name': 'axa220264-my-first-s3',
   'CreationDate': datetime.datetime(2025, 3, 9, 21, 9, 49, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::axa220264-my-first-s3'}],
 'Owner': {'ID': 'c9f5d560e42efca5c00eea655709024bc0e9d08d7fbbd30835a259b38cfb1b4f'}}

In [2]:
import boto3
import pandas as pd
from faker import Faker #fake dataset
import random
from io import StringIO
from datetime import datetime

In [3]:
# Initialize Faker and AWS S3 client
fake = Faker()
s3 = boto3.client('s3')

In [4]:
def generate_cdc_order_data(num_rows=500):
    data = []

    for _ in range(num_rows):
        order = {
            'order_id': fake.uuid4(),
            'customer_id': fake.uuid4(),
            'order_date': fake.date_this_year(),
            'status': random.choice(['CREATED', 'SHIPPED', 'DELIVERED', 'CANCELLED']),
            'product_id': fake.uuid4(),
            'quantity': random.randint(1, 5),
            'price': round(random.uniform(10.0, 500.0), 2),
            'total_amount': 0.0,  # We'll calculate this next
            'cdc_timestamp': datetime.now()  # Simulate CDC timestamp
        }

        order['total_amount'] = round(order['quantity'] * order['price'], 2)
        data.append(order)

    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

In [5]:
# Function to upload data to S3
def upload_to_s3(bucket_name, file_name, df):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    s3.put_object(
        Bucket=bucket_name,
        Key=file_name,
        Body=csv_buffer.getvalue()
    )

    print(f"Data uploaded to s3://{bucket_name}/{file_name}")


# Generate 100 rows of fake CDC order data
df_cdc_order_data = generate_cdc_order_data(num_rows=3000)


# Define S3 bucket and file path
bucket_name = 'axa220264-my-first-s3'
file_name = 'orders/fake_cdc_order_data_1.csv'


# Upload the generated data to S3
upload_to_s3(bucket_name, file_name, df_cdc_order_data)

Data uploaded to s3://axa220264-my-first-s3/orders/fake_cdc_order_data_1.csv


In [6]:
import boto3
s3 = boto3.client('s3')